In [1]:
from datasets import load_dataset

dataset_original = load_dataset("theatticusproject/cuad-qa")

In [2]:
dataset_original

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 22450
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4182
    })
})

In [3]:
import json


print(json.dumps(dataset_original['test'][16], indent=4))

{
    "id": "LohaCompanyltd_20191209_F-1_EX-10.16_11917878_EX-10.16_Supply Agreement__Rofr/Rofo/Rofn",
    "title": "LohaCompanyltd_20191209_F-1_EX-10.16_11917878_EX-10.16_Supply Agreement",
    "context": "Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on , in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the \"order\"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should been written records, only to be valid by buyers and sellers authorized representative signature and confirmation, otherwise will be deemed 

In [4]:
dataset_original['test']['answers']

[{'text': ['SUPPLY CONTRACT'], 'answer_start': [14]},
 {'text': ['The seller:',
   'The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd.'],
  'answer_start': [143, 49]},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': ['The Contract is valid for 5 years, beginning from and ended on .'],
  'answer_start': [10985]},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': ["It will be governed by the law of the People's Republic of China ,otherwise it is governed by United Nations Convention on Contract for the International Sale of Goods."],
  'answer_start': [10691]},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'text': [], 'answer_start': []},
 {'t

In [7]:
# import spacy
from datasets import DatasetDict
import os

num_proc=os.cpu_count()

from more_itertools import windowed
# nlp = spacy.load("en_core_web_sm")

# def chunk_document(document):
#     doc = nlp(document)
#     sentences = [sent.text for sent in doc.sents]
#     return sentences

def process_record(record):
    document_id = "REFERENCE"
    document = record['context']
    answers = record['answers']
    answers = record['answers']
    if len(answers['text']) > 0:
        gold_text = "Highlights:\n" + "\n".join([f"- {answer}" for answer in answers['text']])
    else:
        gold_text = "No Highlights"
    citations = [[document_id, document]]
    # sentences = chunk_document(document)
    # oracle_documents_passages = []
    # # max limit of chunk - tells us number of words, fill in the sentences until max is reached
    # for sentence in windowed(sentences, 5, fillvalue="", step=2):
    #     section = " ".join([s for s in sentence if (s is not None and len(s.strip()) > 0)])
    #     oracle_documents_passages.append([document_id, section])
    
    return {
        'docid': str(record['id']),
        'previous_text': record['question'],
        'gold_text': gold_text,
        'citations': citations,
        # 'oracle_documents_passages': oracle_documents_passages,
    }

dataset_train = dataset_original['train'].select(range(1000))
dataset_test = dataset_original['test']

dataset = DatasetDict({
    'train': dataset_train,
    'test': dataset_test,
})
dataset = dataset.map(process_record, num_proc=num_proc)

Map (num_proc=48):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=48):   0%|          | 0/4182 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.select_columns(['docid', 'previous_text', 'gold_text', 'citations'])
dataset.push_to_hub('ylkhayat/CUAD-generation-workshop', data_dir='data')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ylkhayat/CUAD-generation-workshop/commit/4bf0741a43e3380d63711f9eaa12481f21dcb741', commit_message='Upload dataset', commit_description='', oid='4bf0741a43e3380d63711f9eaa12481f21dcb741', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ylkhayat/CUAD-generation-workshop', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ylkhayat/CUAD-generation-workshop'), pr_revision=None, pr_num=None)

: 

In [24]:
dataset_train = dataset['train'].add_column('answers', dataset_original['train'].select(range(1000))['answers'])
dataset_test = dataset['test'].add_column('answers', dataset_original['test']['answers'])

In [38]:
new_dataset = DatasetDict({
    'train': dataset_train,
    'test': dataset_test,
})


def process_record(record):
    answers = record['answers']
    if len(answers['text']) > 0:
        gold_text = "Highlights:\n" + "\n".join([f"- {answer}" for answer in answers['text']])
    else:
        gold_text = "No Highlights"
    return {
        'gold_text': gold_text,
    }
    
new_dataset = new_dataset.map(process_record, num_proc=num_proc)

Map (num_proc=48):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=48):   0%|          | 0/4182 [00:00<?, ? examples/s]

In [39]:
new_dataset = new_dataset.select_columns(['docid', 'previous_text', 'gold_text', 'citations', 'oracle_documents_passages'])

In [40]:
new_dataset['test'][100]

{'docid': 'DovaPharmaceuticalsInc_20181108_10-Q_EX-10.2_11414857_EX-10.2_Promotion Agreement__Anti-Assignment',
 'previous_text': 'Highlight the parts (if any) of this contract related to "Anti-Assignment" that should be reviewed by a lawyer. Details: Is consent or notice required of a party if the contract is assigned to a third party?',
 'gold_text': "Highlights:\n- Any attempted assignment not in accordance with this Section 13.2 shall be void.\n- Except as provided in this Section 13.2, this Agreement may not be assigned or otherwise transferred, nor may any rights or obligations hereunder be assigned or transferred, by either Party, without the written consent of the other Party (such consent not to be unreasonably withheld); provided that a merger, sale of stock or comparable transaction shall not constitute an assignment.\n- Except to Affiliates of Valeant, Valeant shall not subcontract the Valeant Activities with any Third Party (including any contract sales force).\n- In the e

In [41]:
new_dataset.push_to_hub('ylkhayat/CUAD-generation-workshop', data_dir='data')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/709 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ylkhayat/CUAD-generation-workshop/commit/5988da3a0f40f66e246608ec55881387bb3df39b', commit_message='Upload dataset', commit_description='', oid='5988da3a0f40f66e246608ec55881387bb3df39b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ylkhayat/CUAD-generation-workshop', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ylkhayat/CUAD-generation-workshop'), pr_revision=None, pr_num=None)

In [ ]:
# - complete documents
# text is extracted from context

# - align score (gen and the answer) correctness
#  ( gen and top k) faithfulness
 
#  - we need to break the context into chunks (paragraphs) - normal spacy -> to get sentences and then put in paragraph 
 
 
 
#  - instruct to mention no answer explicitly (challenge)
 
#  - challenge correctness (for no answer, manually state that there is no answer for better scores)
#  - challenge faithfulness (exclude no answers OR base don generations (if exists evaluate faithfulness, otherwise exclude the record))